# Implement Continious Bag of Words Model (CBOW)

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
import numpy as np
import keras.backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Embedding
from keras.utils import np_utils

# Load Data

In [4]:
file = open("Zarathusta.txt", "r")

In [5]:
data = file.read()

In [6]:
data

' When Zarathustra was thirty years old, he left his home and the lake of his home, and went into the mountains. There he enjoyed his spirit and solitude, and for ten years did not weary of it. But at last his heart changed,—and rising one morning with the rosy dawn, he went before the sun, and spake thus unto it:\n\nThou great star! What would be thy happiness if thou hadst not those for whom thou shinest!\n\nFor ten years hast thou climbed hither unto my cave: thou wouldst have wearied of thy light and of the journey, had it not been for me, mine eagle, and my serpent.\n\nBut we awaited thee every morning, took from thee thine overflow and blessed thee for it.\n\nLo! I am weary of my wisdom, like the bee that hath gathered too much honey; I need hands outstretched to take it.\n\nI would fain bestow and distribute, until the wise have once more become joyous in their folly, and the poor happy in their riches.\n\nTherefore must I descend into the deep: as thou doest in the evening, whe

In [7]:
words = data.split()

In [8]:
words = [w.lower().replace("!","").replace(",","").replace(".","").replace("\n", "") for w in words]

In [9]:
words

['when',
 'zarathustra',
 'was',
 'thirty',
 'years',
 'old',
 'he',
 'left',
 'his',
 'home',
 'and',
 'the',
 'lake',
 'of',
 'his',
 'home',
 'and',
 'went',
 'into',
 'the',
 'mountains',
 'there',
 'he',
 'enjoyed',
 'his',
 'spirit',
 'and',
 'solitude',
 'and',
 'for',
 'ten',
 'years',
 'did',
 'not',
 'weary',
 'of',
 'it',
 'but',
 'at',
 'last',
 'his',
 'heart',
 'changed—and',
 'rising',
 'one',
 'morning',
 'with',
 'the',
 'rosy',
 'dawn',
 'he',
 'went',
 'before',
 'the',
 'sun',
 'and',
 'spake',
 'thus',
 'unto',
 'it:',
 'thou',
 'great',
 'star',
 'what',
 'would',
 'be',
 'thy',
 'happiness',
 'if',
 'thou',
 'hadst',
 'not',
 'those',
 'for',
 'whom',
 'thou',
 'shinest',
 'for',
 'ten',
 'years',
 'hast',
 'thou',
 'climbed',
 'hither',
 'unto',
 'my',
 'cave:',
 'thou',
 'wouldst',
 'have',
 'wearied',
 'of',
 'thy',
 'light',
 'and',
 'of',
 'the',
 'journey',
 'had',
 'it',
 'not',
 'been',
 'for',
 'me',
 'mine',
 'eagle',
 'and',
 'my',
 'serpent',
 'but',


In [10]:
vocab = set(words)

In [11]:
vocab_size = len(vocab)

In [12]:
vocab_size

165

In [13]:
words_to_idx = {word:i for i,word in enumerate(vocab)}
idx_to_words = {i:word for i,word in enumerate(vocab)}

In [14]:
words_to_idx

{'mountains': 0,
 'i': 1,
 'mine': 2,
 'bee': 3,
 'itself': 4,
 'weary': 5,
 'must': 6,
 'man': 7,
 'his': 8,
 'hither': 9,
 'go': 10,
 'great': 11,
 'descend': 12,
 'serpent': 13,
 'did': 14,
 'it:': 15,
 'star': 16,
 'hath': 17,
 'light': 18,
 'may': 19,
 'for': 20,
 'eagle': 21,
 'one': 22,
 'outstretched': 23,
 'thee': 24,
 'me': 25,
 'about': 26,
 'without': 27,
 'become': 28,
 'riches': 29,
 'much': 30,
 'thou': 31,
 'at': 32,
 'this': 33,
 'that': 34,
 'joyous': 35,
 'greatest': 36,
 'wearied': 37,
 'am': 38,
 'flow': 39,
 'it': 40,
 'hands': 41,
 'was': 42,
 'honey;': 43,
 'bless': 44,
 'had': 45,
 'even': 46,
 'left': 47,
 'have': 48,
 'home': 49,
 'there': 50,
 'enjoyed': 51,
 'take': 52,
 'tranquil': 53,
 'dawn': 54,
 'lake': 55,
 'would': 56,
 'those': 57,
 'shall': 58,
 'what': 59,
 'as': 60,
 'went': 61,
 'to': 62,
 'morning': 63,
 'happy': 64,
 'climbed': 65,
 'a': 66,
 'bestow': 67,
 'thy': 68,
 'behold': 69,
 'ten': 70,
 'years': 71,
 'spake': 72,
 'spirit': 73,
 'dist

In [15]:
idx_to_words

{0: 'mountains',
 1: 'i',
 2: 'mine',
 3: 'bee',
 4: 'itself',
 5: 'weary',
 6: 'must',
 7: 'man',
 8: 'his',
 9: 'hither',
 10: 'go',
 11: 'great',
 12: 'descend',
 13: 'serpent',
 14: 'did',
 15: 'it:',
 16: 'star',
 17: 'hath',
 18: 'light',
 19: 'may',
 20: 'for',
 21: 'eagle',
 22: 'one',
 23: 'outstretched',
 24: 'thee',
 25: 'me',
 26: 'about',
 27: 'without',
 28: 'become',
 29: 'riches',
 30: 'much',
 31: 'thou',
 32: 'at',
 33: 'this',
 34: 'that',
 35: 'joyous',
 36: 'greatest',
 37: 'wearied',
 38: 'am',
 39: 'flow',
 40: 'it',
 41: 'hands',
 42: 'was',
 43: 'honey;',
 44: 'bless',
 45: 'had',
 46: 'even',
 47: 'left',
 48: 'have',
 49: 'home',
 50: 'there',
 51: 'enjoyed',
 52: 'take',
 53: 'tranquil',
 54: 'dawn',
 55: 'lake',
 56: 'would',
 57: 'those',
 58: 'shall',
 59: 'what',
 60: 'as',
 61: 'went',
 62: 'to',
 63: 'morning',
 64: 'happy',
 65: 'climbed',
 66: 'a',
 67: 'bestow',
 68: 'thy',
 69: 'behold',
 70: 'ten',
 71: 'years',
 72: 'spake',
 73: 'spirit',
 74: '

# Generate Context-Target Pairs

In [16]:
c = 2
dim = 100

In [17]:
X_train = []
Y_train = []

for index, i in enumerate(words):
    
    if(index >= len(words)-2) or (index <= 1):
        continue
    start = index -c
    end = index + c + 1
    
    context = words[start:end]
    context.remove(i)
    target = i
    
    X_train.append([words_to_idx[w] for w in context])
    Y_train.append([words_to_idx[target]])
    
    

In [18]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [19]:
Y_train = np_utils.to_categorical(Y_train, vocab_size)

# Train Model

In [21]:
cbow = Sequential()

In [22]:
cbow.add(Embedding(input_dim=vocab_size, output_dim=dim, input_length=c*2))

In [23]:
cbow.add(Lambda(lambda x : K.mean(x, axis=1), output_shape=(dim,)))

In [24]:
cbow.add(Dense(vocab_size, activation="softmax"))

In [25]:
cbow.compile(optimizer=SGD(), loss="categorical_crossentropy", metrics=["accuracy"])

In [26]:
cbow.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            16500     
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 165)               16665     
                                                                 
Total params: 33,165
Trainable params: 33,165
Non-trainable params: 0
_________________________________________________________________


In [27]:
cbow.fit(X_train,Y_train,epochs=5000)

Epoch 1/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1063 - accuracy: 0.0037
Epoch 2/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1056 - accuracy: 0.0037
Epoch 3/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1048 - accuracy: 0.0037
Epoch 4/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1040 - accuracy: 0.0073
Epoch 5/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1033 - accuracy: 0.0147
Epoch 6/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1025 - accuracy: 0.0183
Epoch 7/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1018 - accuracy: 0.0183
Epoch 8/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1010 - accuracy: 0.0256
Epoch 9/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1003 - accuracy: 0.0293
Epoch 10/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0995 - accuracy: 0.0366
Epoch 11/5000
9/9 [

9/9 [==============================] - 0s 2ms/step - loss: 5.0471 - accuracy: 0.0586
Epoch 84/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0464 - accuracy: 0.0586
Epoch 85/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0457 - accuracy: 0.0586
Epoch 86/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0450 - accuracy: 0.0586
Epoch 87/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0443 - accuracy: 0.0586
Epoch 88/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0436 - accuracy: 0.0586
Epoch 89/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0429 - accuracy: 0.0586
Epoch 90/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0422 - accuracy: 0.0586
Epoch 91/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0415 - accuracy: 0.0586
Epoch 92/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0408 - accuracy: 0.0586
Epoch 93/5000
9/9 [=====

9/9 [==============================] - 0s 2ms/step - loss: 4.9934 - accuracy: 0.0586
Epoch 166/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9928 - accuracy: 0.0586
Epoch 167/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9922 - accuracy: 0.0586
Epoch 168/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9916 - accuracy: 0.0586
Epoch 169/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9910 - accuracy: 0.0586
Epoch 170/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9904 - accuracy: 0.0586
Epoch 171/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9898 - accuracy: 0.0586
Epoch 172/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9892 - accuracy: 0.0586
Epoch 173/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9886 - accuracy: 0.0586
Epoch 174/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9880 - accuracy: 0.0586
Epoch 175/5000


9/9 [==============================] - 0s 2ms/step - loss: 4.9464 - accuracy: 0.0586
Epoch 248/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9459 - accuracy: 0.0586
Epoch 249/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9454 - accuracy: 0.0586
Epoch 250/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9449 - accuracy: 0.0586
Epoch 251/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9443 - accuracy: 0.0586
Epoch 252/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9438 - accuracy: 0.0586
Epoch 253/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9433 - accuracy: 0.0586
Epoch 254/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9428 - accuracy: 0.0586
Epoch 255/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9423 - accuracy: 0.0586
Epoch 256/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9418 - accuracy: 0.0586
Epoch 257/5000


9/9 [==============================] - 0s 2ms/step - loss: 4.9063 - accuracy: 0.0586
Epoch 330/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9058 - accuracy: 0.0586
Epoch 331/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9054 - accuracy: 0.0586
Epoch 332/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9050 - accuracy: 0.0586
Epoch 333/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9045 - accuracy: 0.0586
Epoch 334/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9041 - accuracy: 0.0586
Epoch 335/5000
9/9 [==============================] - 0s 1ms/step - loss: 4.9036 - accuracy: 0.0586
Epoch 336/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9032 - accuracy: 0.0586
Epoch 337/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9028 - accuracy: 0.0586
Epoch 338/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9023 - accuracy: 0.0586
Epoch 339/5000


...

9/9 [==============================] - 0s 2ms/step - loss: 3.1484 - accuracy: 0.3187
Epoch 4631/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1476 - accuracy: 0.3187
Epoch 4632/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1469 - accuracy: 0.3187
Epoch 4633/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1462 - accuracy: 0.3187
Epoch 4634/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1454 - accuracy: 0.3187
Epoch 4635/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1447 - accuracy: 0.3187
Epoch 4636/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1439 - accuracy: 0.3187
Epoch 4637/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1432 - accuracy: 0.3187
Epoch 4638/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1424 - accuracy: 0.3187
Epoch 4639/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1417 - accuracy: 0.3187
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 3.0882 - accuracy: 0.3223
Epoch 4712/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0875 - accuracy: 0.3223
Epoch 4713/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0867 - accuracy: 0.3223
Epoch 4714/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0860 - accuracy: 0.3223
Epoch 4715/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0853 - accuracy: 0.3223
Epoch 4716/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0845 - accuracy: 0.3223
Epoch 4717/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0838 - accuracy: 0.3223
Epoch 4718/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0830 - accuracy: 0.3223
Epoch 4719/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0823 - accuracy: 0.3223
Epoch 4720/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0816 - accuracy: 0.3223
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 3.0280 - accuracy: 0.3333
Epoch 4793/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0272 - accuracy: 0.3333
Epoch 4794/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0265 - accuracy: 0.3333
Epoch 4795/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0258 - accuracy: 0.3333
Epoch 4796/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0250 - accuracy: 0.3297
Epoch 4797/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0243 - accuracy: 0.3333
Epoch 4798/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0235 - accuracy: 0.3333
Epoch 4799/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0228 - accuracy: 0.3333
Epoch 4800/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0220 - accuracy: 0.3333
Epoch 4801/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0213 - accuracy: 0.3333
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 2.9677 - accuracy: 0.3407
Epoch 4874/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9669 - accuracy: 0.3407
Epoch 4875/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9662 - accuracy: 0.3407
Epoch 4876/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9654 - accuracy: 0.3407
Epoch 4877/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9647 - accuracy: 0.3407
Epoch 4878/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9639 - accuracy: 0.3407
Epoch 4879/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9632 - accuracy: 0.3407
Epoch 4880/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9624 - accuracy: 0.3407
Epoch 4881/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9617 - accuracy: 0.3407
Epoch 4882/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9609 - accuracy: 0.3407
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 2.9074 - accuracy: 0.3407
Epoch 4955/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9066 - accuracy: 0.3407
Epoch 4956/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9059 - accuracy: 0.3407
Epoch 4957/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9051 - accuracy: 0.3407
Epoch 4958/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9044 - accuracy: 0.3407
Epoch 4959/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9036 - accuracy: 0.3407
Epoch 4960/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9029 - accuracy: 0.3407
Epoch 4961/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9022 - accuracy: 0.3443
Epoch 4962/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9014 - accuracy: 0.3443
Epoch 4963/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9007 - accuracy: 0.3407
Epoch 

In [28]:
pred = cbow.predict(X_train)

9/9 [==============================] - 0s 1ms/step


In [29]:
pred = pred.argmax(axis=1)

In [30]:
pred

array([ 31,  31,  71,  71, 101, 109,   8, 106, 109, 130, 109, 109, 130,
       130, 109, 109,  79, 130,  61,  61, 101, 109,   8, 109, 109,   8,
       109,  20,  70,  71,  20,  40,   5, 106,  40,  40, 106, 106,   8,
       109, 109, 130, 130,  34,  34, 130,  61,  61, 130,  61, 109, 130,
       109, 109, 130, 130,  31,  16,  31,  31,  16,  31,  31,   1,  31,
       135,  16,  31,  20,  71,  31,  20,  71,  31,  20,  20,  31,  71,
        20,  31,  31,  31,  31,  31,  31,  31,  16, 100, 109, 106,  68,
        18, 109, 106, 130, 106, 106,  40, 139,  20,  20,  31,  70,   8,
       109,   8, 106, 109,   1,   1,  24,  24,  24,  24,  24, 109, 130,
       109, 109,  24,  24,   1,  40, 124, 106,  40,  40, 106, 130, 109,
       106, 130,  34,  34, 130, 130,   1,   6,   1,   1,  62,  62,   1,
         1,  62,  40,   1,   1,  40, 130, 109, 109,  61, 130, 109,  34,
       130,  31,  31,  31, 109, 130,  61, 109, 130,  61, 109, 130, 130,
         1,   6,   6,   1,  12,  79, 130, 108, 108, 130, 108, 10

In [32]:
i = 0
for context in X_train:
    print("==================Context================")
    for w in context:
        print(idx_to_words[w])
    print("=================Predicted================")
    print(idx_to_words[pred[i]])
    i+=1

==================Context================
when
zarathustra
thirty
years
=================Predicted================
thou
==================Context================
zarathustra
was
years
old
=================Predicted================
thou
==================Context================
was
thirty
old
he
=================Predicted================
years
==================Context================
thirty
years
he
left
=================Predicted================
years
==================Context================
years
old
left
his
=================Predicted================
he
==================Context================
old
he
his
home
=================Predicted================
and
==================Context================
he
left
home
and
=================Predicted================
his
==================Context================
left
his
and
the
=================Predicted================
of
==================Context================
his
home
the
lake
=================Predicted================
and
=============